In [42]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import scipy


# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session
%config Completer.use_jedi = False

/kaggle/input/digit-recognizer/sample_submission.csv
/kaggle/input/digit-recognizer/train.csv
/kaggle/input/digit-recognizer/test.csv


In [4]:
%config Completer.use_jedi = False #For Autocomplete


In [102]:
trainingData = pd.read_csv('/kaggle/input/digit-recognizer/train.csv')
# testData = pd.read_csv('/kaggle/input/digit-recognizer/test.csv')


In [103]:

trainingData = (np.array(trainingData))
# testData = (np.array(testData))

np.random.shuffle(trainingData)
# np.random.shuffle(testData)

trainingData = trainingData.T
# testData = testData.T

testData = trainingData[: , :1000]
trainingData = trainingData[: , 1000:]


n, m = trainingData.shape

trainingTargets = trainingData[0,:]
trainingPredictors = trainingData[1:,:]
trainingPredictors = trainingPredictors/255

testTargets = testData[0,:]
testPredictors = testData[1:,:]
testPredictors = testPredictors/255


In [123]:
class NeuralNetwork:
    def __init__(self, predictors, target, hiddenLayerNodes):
        self.predictors = predictors
        self.target = target
        self.n0, self.m = self.predictors.shape
        self.n1 = hiddenLayerNodes
        self.n2 = 10
        self.W1, self.B1, self.W2, self.B2 = [None] * 4 
        self.A1, self.Z1, self.A2, self.Z2 = [None] * 4
    
    def gradientDescent(self, iterations, alpha):
        self.initialiseWeightsAndBiases()
        
        for iteration in range(iterations):
            self.updateWeightsAndBiases(alpha)
            if(iteration%10 == 0):
#                 print("W1 = ",self.W1)
#                 print("W2 = ",self.W2)
#                 print("B1 = ",self.B1)
#                 print("B2 = ",self.B2)
                
                print(str(iteration) + " " + str(self.getAccuracy()))
            
    def initialiseWeightsAndBiases(self):
        self.W1 = np.random.rand(10, 784)-0.5
        self.W2 = np.random.rand(10, 10)-0.5
        self.B1 = np.zeros((10,1))-0.5
        self.B2 = np.zeros((10,1))-0.5
    
    def updateWeightsAndBiases(self, alpha):
        self.forwardPropagation()
        Y = self.getOneHot()
        
        DZ2 = self.A2 - Y
        DW2 = (1/self.m)*DZ2.dot(self.A1.T)
        DB2 = (1/self.m)*np.sum(DZ2,1).reshape(-1, 1)
        
        DZ1 = self.W2.T.dot(DZ2)*self.derivativeOfA1()
        DW1 = (1/self.m)*DZ1.dot(self.predictors.T)
        DB1 = (1/self.m)*np.sum(DZ1,1).reshape(-1, 1)
        
        self.W2 = self.W2 - alpha*DW2
        self.W1 = self.W1 - alpha*DW1
        self.B2 = self.B2 - alpha*DB2
        self.B1 = self.B1 - alpha*DB1
    
    def forwardPropagation(self):
        self.Z1 = self.W1.dot(self.predictors) + self.B1
        self.A1 = self.ReLU(self.Z1)
        self.Z2 = self.W2.dot(self.A1) + self.B2
        self.A2 = self.softmax(self.Z2 )
        
    def ReLU(self,Z):
        return np.maximum(Z, 0)
    
    def softmax(self,Z):
#         Z = Z- np.max(Z,axis=0)
        return np.exp(Z)/np.sum(np.exp(Z), axis = 0)
    
    def getOneHot(self):
        Y = np.zeros((10, self.m))
        Y[self.target, np.arange(self.m)] = 1
        return Y
    
    def derivativeOfA1(self):
        return self.Z1>0
    
    def getPredictions(self,Y):
        return np.argmax(Y, 0)
    
    def getAccuracy(self):
        print("predictions" ,self.getPredictions(self.A2))
        return np.sum(self.getPredictions(self.A2)==self.target)/(self.target.size)
    
    def calculateTestAccuracy(self, testPredictors, testTarget):
        self.predictors = testPredictors
        self.target = testTarget
        
        self.forwardPropagation()
        
        self.getAccuracy()
        
    def predictRandomExample(self):
        randomIndex = np.random.randint(0, self.predictors.shape[1])
        example = self.predictors[:, randomIndex].reshape(28, 28)
        
        plt.gray()
        plt.imshow(example)
        
        print("Target: ", self.target[randomIndex], " ")
        print("Prediction: ", self.getPredictions( self.A2[:, randomIndex]), " ")


In [ ]:
nn =  NeuralNetwork(trainingPredictors, trainingTargets, 10 )
nn.gradientDescent(500, 0.1)


predictions [6 3 0 ... 8 8 8]
0 0.12982926829268293
predictions [6 3 1 ... 6 8 0]
10 0.16819512195121952
predictions [6 0 1 ... 6 8 0]
20 0.21646341463414634
predictions [1 0 1 ... 6 8 0]
30 0.26234146341463416
predictions [1 0 1 ... 6 8 0]
40 0.30221951219512194
predictions [1 0 1 ... 6 8 0]
50 0.3421219512195122
predictions [3 0 6 ... 6 8 3]
60 0.3840487804878049
predictions [0 7 6 ... 6 8 8]
70 0.4224390243902439
predictions [7 7 6 ... 1 8 8]
80 0.46451219512195124
predictions [7 7 6 ... 1 8 8]
90 0.5137073170731707
predictions [7 7 4 ... 1 8 8]
100 0.5584146341463415
predictions [7 7 4 ... 1 8 8]
110 0.5934634146341463
predictions [7 7 4 ... 1 8 8]
120 0.6228292682926829
predictions [7 7 4 ... 1 8 8]
130 0.6445365853658537
predictions [7 7 6 ... 1 8 8]
140 0.6627560975609756
predictions [7 7 6 ... 1 8 8]
150 0.6775853658536586
predictions [7 7 6 ... 1 8 8]
160 0.6901951219512195
predictions [7 7 6 ... 1 8 8]
170 0.7020731707317073
predictions [7 7 6 ... 1 8 8]
180 0.711268292682926

In [ ]:
nn.calculateTestAccuracy( testPredictors, testTargets)
nn.predictRandomExample()
